In [1]:
!wget https://aveit.s3.amazonaws.com/higlass/bam/example_higlass.bam -O example.bam
!wget https://aveit.s3.amazonaws.com/higlass/bam/example_higlass.bam.bai -O example.bam.bai
!touch example.bam.bai # ensures no errors from pysam

--2023-07-15 10:21:35--  https://aveit.s3.amazonaws.com/higlass/bam/example_higlass.bam
Resolving aveit.s3.amazonaws.com (aveit.s3.amazonaws.com)... 52.216.32.33, 52.216.61.225, 3.5.10.213, ...
Connecting to aveit.s3.amazonaws.com (aveit.s3.amazonaws.com)|52.216.32.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4993966 (4.8M) [binary/octet-stream]
Saving to: ‘example.bam’

example.bam         100%[===================>]   4.76M  6.67MB/s    in 0.7s    

2023-07-15 10:21:36 (6.67 MB/s) - ‘example.bam’ saved [4993966/4993966]

--2023-07-15 10:21:36--  https://aveit.s3.amazonaws.com/higlass/bam/example_higlass.bam.bai
Resolving aveit.s3.amazonaws.com (aveit.s3.amazonaws.com)... 52.216.32.33, 52.216.61.225, 3.5.10.213, ...
Connecting to aveit.s3.amazonaws.com (aveit.s3.amazonaws.com)|52.216.32.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28176 (28K) [binary/octet-stream]
Saving to: ‘example.bam.bai’

example.bam.bai     100%[====

In [1]:
import pathlib
import io

import pysam
import polars as pl
import pyarrow as pa
import pandas as pd
import bioframe

import dask_ngs



In [2]:
from io import BytesIO

import bioframe
import dask
import dask.dataframe as dd
import oxbow as ox
import pandas as pd
import pyarrow.ipc


def _read_bam_query_from_path(
    path: str, chrom: str, start: int, end: int
) -> pd.DataFrame:
    stream = BytesIO(ox.read_bam(path, f"{chrom}:{start}-{end}"))
    ipc = pyarrow.ipc.open_file(stream)
    return ipc.read_pandas()

In [105]:
from gzip import GzipFile
from io import BytesIO

import numpy as np
import pandas as pd

BAI_MIN_SHIFT = 14
BAI_DEPTH = 5
COMPRESSED_POSITION_SHIFT = 16
UNCOMPRESSED_POSITION_MASK = 0xffff
BLOCKSIZE = 65536

def read_bai(path):
    """
    https://samtools.github.io/hts-specs/SAMv1.pdf
    """
    int_kwargs = {'byteorder': 'little', 'signed': False}
    f = open(path, "rb")
    # read the 4-byte magic number
    magic = f.read(4)

    # read the number of reference sequences
    n_ref = int.from_bytes(f.read(4), **int_kwargs)

    # read the reference sequence indices
    references = []
    for i in range(n_ref):
        ref = {'ref_id': i}

        # The "Bin Index"
        chunks = []
        n_bin = int.from_bytes(f.read(4), **int_kwargs)
        for _ in range(n_bin):
            # bin number
            bin_id = int.from_bytes(f.read(4), **int_kwargs)

            if bin_id == 37450:
                # This is an entry that describes the "pseudo-bin" for the 
                # reference, using the same byte layout as normal bins but 
                # interpreted differently.
                # The ref beg/ref end fields locate the first and last reads on
                # this reference sequence, whether they are mapped or placed 
                # unmapped. Thus they are equal to the minimum chunk beg and 
                # maximum chunk end respectively.
                n_chunk = int.from_bytes(f.read(4), **int_kwargs)  # always 2
                # Not really a chunk
                vpos = int.from_bytes(f.read(8), **int_kwargs)
                ref["ref_beg.cpos"] = vpos >> COMPRESSED_POSITION_SHIFT
                ref["ref_beg.upos"] = vpos & UNCOMPRESSED_POSITION_MASK
                vpos = int.from_bytes(f.read(8), **int_kwargs)
                ref["ref_end.cpos"] = vpos >> COMPRESSED_POSITION_SHIFT
                ref["ref_end.upos"] = vpos & UNCOMPRESSED_POSITION_MASK
                # Not really a chunk
                ref["n_mapped"] = int.from_bytes(f.read(8), **int_kwargs)
                ref["n_unmapped"] = int.from_bytes(f.read(8), **int_kwargs)
                continue

            n_chunk = int.from_bytes(f.read(4), **int_kwargs)
            for _ in range(n_chunk):
                vpos = int.from_bytes(f.read(8), **int_kwargs)
                chunk_beg_cpos = vpos >> COMPRESSED_POSITION_SHIFT
                chunk_beg_upos = vpos & UNCOMPRESSED_POSITION_MASK
                vpos = int.from_bytes(f.read(8), **int_kwargs)
                chunk_end_cpos = vpos >> COMPRESSED_POSITION_SHIFT
                chunk_end_upos = vpos & UNCOMPRESSED_POSITION_MASK

                chunks.append((bin_id, chunk_beg_cpos, chunk_beg_upos, chunk_end_cpos, chunk_end_upos))   

            ref["bins"] = chunks

        # The "Linear Index"
        ref["ioffsets"] = []
        n_intv = int.from_bytes(f.read(4), **int_kwargs)
        for _ in range(n_intv):
            vpos = int.from_bytes(f.read(8), **int_kwargs)
            ioffset_cpos = vpos >> COMPRESSED_POSITION_SHIFT
            ioffset_upos = vpos & UNCOMPRESSED_POSITION_MASK
            ref["ioffsets"].append((ioffset_cpos, ioffset_upos))

        references.append(ref)

        # Check for the optional n_no_coor at the end of the file
        try:
            n_no_coor = int.from_bytes(f.read(8), **int_kwargs)
        except:
            n_no_coor = None

    return references, n_no_coor       


In [111]:
   
# Convert the deconstructed virtual positions into dataframes
def make_dataframe(references: pd.DataFrame) -> pd.DataFrame:

    for ref in references:
        if not 'bins' in ref:
            continue

        ref["bins"] = pd.DataFrame(
            ref["bins"],
            columns=["bin_id", "chunk_beg.cpos", "chunk_beg.upos", "chunk_end.cpos", "chunk_end.upos"]
        )
        ref["ioffsets"] = pd.DataFrame(
            ref["ioffsets"],
            columns=["ioffset.cpos", "ioffset.upos"]
        )

    return references

bai = read_bai('example.bam.bai')
ref = bai[0]
ref['bins']
df = make_dataframe(bai)
df

{'ref_id': 0,
 'bins': [(585, 157643, 61968, 163802, 9643),
  (585, 456717, 19251, 456717, 20088),
  (73, 720450, 14227, 720450, 27457),
  (73, 1825729, 40851, 1832074, 276),
  (73, 3306112, 550, 3306112, 11849),
  (73, 4929731, 62149, 4937687, 11573),
  (586, 941494, 61277, 941494, 64577),
  (586, 1098225, 39987, 1098225, 45238),
  (586, 1273813, 11833, 1273813, 18402),
  (586, 1633523, 37951, 1633523, 42945),
  (587, 2239862, 10042, 2239862, 22158),
  (587, 2496504, 42100, 2528450, 41584),
  (587, 2812217, 30007, 2812217, 42248),
  (587, 3063110, 42982, 3063110, 56779),
  (588, 3537795, 15096, 3537795, 25731),
  (588, 3778303, 19870, 3778303, 34495),
  (588, 3985112, 7419, 3985112, 14925),
  (588, 4168773, 61270, 4175801, 5793),
  (588, 4358871, 33608, 4358871, 47638),
  (588, 4543124, 9358, 4543124, 17387),
  (588, 4731348, 36165, 4731348, 47217),
  (4686, 38660, 0, 157643, 61968),
  (4687, 163802, 9643, 456717, 19251),
  (4688, 456717, 20088, 720450, 14227),
  (4689, 720450, 27457,

In [97]:
offsets = ref['ioffsets']['ioffset.cpos']

#o = offsets[:, 1:,] - offsets[:,:-1]
diffs = [0] + [y - x for x, y in zip(offsets, offsets[1:])]
#ref['ioffsets'].append(diffs)  
ref['ioffsets']['ioffset.cpos.diff'] = diffs
offsets = ref['ioffsets']
offsets

,ioffset.cpos,ioffset.upos,ioffset.cpos.diff,chunk_id
0,38660,0,0,0
1,38660,0,0,0
2,38660,0,0,0
3,38660,0,0,0
4,38660,0,0,0
5,38660,0,0,0
6,157643,61968,118983,0
7,456717,19251,299074,0
8,720450,14227,263733,0
9,941494,61277,221044,0


Write a function that creates a list of consolidated chunks each having a size no greater than an input (chunk size)

Outputs a list of start position and end position. Keep the uncompressed and compressed information in the function.

In [103]:
# TODO: Make this more Pythonic

def get_chunk_info(offsets, chunksize: int):
    index = 0
    sum = 0
    start_bytes = 0
    for i in range(0, len(offsets)):
        diff = offsets.iloc[i,2] # 2 refers to the diff column
        sum += diff
        start_bytes += diff
        offsets.at[i,'start_bytes'] = start_bytes
        if sum > chunksize:
            index += 1
            sum = 0
        offsets.at[i,'chunk_id'] = index
    return offsets

o = get_chunk_info(offsets, 1_000_000)
o

,ioffset.cpos,ioffset.upos,ioffset.cpos.diff,chunk_id,start_bytes
0,38660,0,0,0,0.0
1,38660,0,0,0,0.0
2,38660,0,0,0,0.0
3,38660,0,0,0,0.0
4,38660,0,0,0,0.0
5,38660,0,0,0,0.0
6,157643,61968,118983,0,118983.0
7,456717,19251,299074,0,418057.0
8,720450,14227,263733,0,681790.0
9,941494,61277,221044,0,902834.0


In [126]:
df = o.iloc[:, :4].copy()
df2 = df.groupby("ioffset.cpos").agg({
    "ioffset.upos": "first",
    "ioffset.cpos.diff": "first"
}).reset_index()
df2

,ioffset.cpos,ioffset.upos,ioffset.cpos.diff
0,38660,0,0
1,157643,61968,118983
2,456717,19251,299074
3,720450,14227,263733
4,941494,61277,221044
5,1098225,39987,156731
6,1273813,11833,175588
7,1633523,37951,359710
8,1751276,63981,117753
9,1825729,40851,74453


In [138]:
# TODO: Make this more Pythonic

def get_chunk_info(offsets: pd.DataFrame, chunksize: int):
    index = 0
    sum = 0
    start_bytes = 0
    colidx = offsets.columns.get_loc("ioffset.cpos.diff")
    for i in range(0, len(offsets)):        
        diff = offsets.iloc[i,colidx]
        sum += diff
        start_bytes += diff
        offsets.at[i,'start_bytes'] = int(start_bytes)
        if sum > chunksize:
            index += 1
            sum = 0
        offsets.at[i,'chunk_id'] = int(index)
    offsets['start_bytes'] = offsets['start_bytes'].astype(int)
    offsets['chunk_id'] = offsets['chunk_id'].astype(int)
    return offsets

off = df2.copy()
o = get_chunk_info(off, 1_000_000)
o

,ioffset.cpos,ioffset.upos,ioffset.cpos.diff,start_bytes,chunk_id
0,38660,0,0,0,0
1,157643,61968,118983,118983,0
2,456717,19251,299074,418057,0
3,720450,14227,263733,681790,0
4,941494,61277,221044,902834,0
5,1098225,39987,156731,1059565,1
6,1273813,11833,175588,1235153,1
7,1633523,37951,359710,1594863,1
8,1751276,63981,117753,1712616,1
9,1825729,40851,74453,1787069,1


In [3]:
chromsizes = bioframe.fetch_chromsizes("hg38")
chunksize = 10_000_000
chunk_spans = bioframe.binnify(chromsizes, chunksize)
chunk_spans["start"] += 1
chunk_spans

,chrom,start,end
0,chr1,1,10000000
1,chr1,10000001,20000000
2,chr1,20000001,30000000
3,chr1,30000001,40000000
4,chr1,40000001,50000000
...,...,...,...
318,chrY,20000001,30000000
319,chrY,30000001,40000000
320,chrY,40000001,50000000
321,chrY,50000001,57227415


In [104]:
path = "example.bam"
chunks = [
    dask.delayed(_read_bam_query_from_path)(path, chrom, start, end)
    for chrom, start, end in chunk_spans.to_numpy()
]
chunks
#df = dd.from_delayed(chunks)
#df

[Delayed('_read_bam_query_from_path-a05ab1ba-dd16-4b86-bb23-5e1763e278d8'),
 Delayed('_read_bam_query_from_path-191e4c27-55b0-45a8-b198-416355bad11c'),
 Delayed('_read_bam_query_from_path-83bab78a-deac-4bf5-a3de-d86d720fb785'),
 Delayed('_read_bam_query_from_path-b7616975-1b15-4cbc-ade3-9abf149a02d9'),
 Delayed('_read_bam_query_from_path-4e5571d5-d357-4164-b446-eb64220eab47'),
 Delayed('_read_bam_query_from_path-dd0ae847-f836-4313-b47f-d698eea081a6'),
 Delayed('_read_bam_query_from_path-2ac8385c-82a1-4e74-917a-079632e33d98'),
 Delayed('_read_bam_query_from_path-e66b6bfd-456e-4c21-9d30-e136ecf4274c'),
 Delayed('_read_bam_query_from_path-5e0ddfb7-96b4-44dc-9c19-e5683fb20fb6'),
 Delayed('_read_bam_query_from_path-0efa61fc-7637-4142-a27f-af3a69d540d4'),
 Delayed('_read_bam_query_from_path-43fcc793-95e6-4c94-a160-b82d51c55e7f'),
 Delayed('_read_bam_query_from_path-ddf25d82-6587-4de3-8da1-f8cc749cb24f'),
 Delayed('_read_bam_query_from_path-01e77eb0-df96-47b9-a140-3eacbe82ec0d'),
 Delayed('_r

In [10]:
df.partitions[0].compute()

,qname,flag,rname,pos,mapq,cigar,rnext,pnext,tlen,seq,qual,end
0,SRR4435251::::313654063,83,chr1,82736,0,100M,chr1,82517,-319,TAAAAAAGAATGCAGATATTACAAAACCAGTTTACAAAAGTTACTA...,??????????????????????????????????????????????...,82835
1,SRR4435251::::1356039,147,chr1,82742,0,100M,chr1,82511,-331,AGAATGCAGATATTACAAAACCAGTTTACAAAAGTTACTAAACAAA...,??????????????????????????????????????????????...,82841
2,SRR4435251::::270890793,163,chr1,82744,0,100M,chr1,82982,338,AATGCAGATATTACAAAACCAGTTTACAAAAGTTACTAAACAAATA...,??????????????????????????????????????????????...,82843
3,SRR4435251::::1356040,83,chr1,82748,0,100M,chr1,82488,-360,CAGATATTACAAAACCAGTTTACAAAAGTTACTAAACAAATAAAAA...,??????????????????????????????????????????????...,82847
4,SRR4435251::::1482538,147,chr1,82749,0,100M,chr1,82506,-343,AGATATTACAAAACCAGTTTACAAAAGTTACTAAACAAATAAAAAC...,??????????????????????????????????????????????...,82848
...,...,...,...,...,...,...,...,...,...,...,...,...
160173,SRR4435251::::1378461,147,chr1,528790,0,100M,chr1,528599,-291,CTTGTCTGGTCTGGCTCTGCCCCACTCTCCTTCTCACCTAGTTGGA...,??????????????????????????????????????????????...,528889
160174,SRR4435251::::455831805,83,chr1,528792,0,100M,chr1,528483,-409,TGTCTGGTCTGGCTCTGCCCCACTCTCCTTCTCACCTAGTTGGAAT...,??????????????????????????????????????????????...,528891
160175,SRR4435251::::1378464,83,chr1,528800,0,100M,chr1,528695,-205,CTGGCTCTGCCCCACTCTCCTTCTCTCCTAGTTGGAATCCCTAGCT...,??5????????5?5???5'55+???+55????5??5??55????5?...,528899
160176,SRR4435251::::1378465,147,chr1,528803,0,100M,chr1,528567,-336,GCTCTGCCCCACTCTCCTTCTCACCTAGTTGGAATCCCTAGCTACT...,????????5?5???????????????????????????????????...,528902


In [3]:
file = pathlib.Path("example.bam")
region = ("chr1", 1, 8_000_000)

In [4]:
%timeit pysam_run(file, *region)

2.08 s ± 26.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit oxbow_polars(file, *region)

1.2 s ± 19.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%timeit oxbow_pandas(file, *region)

1.24 s ± 37.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
